In [1]:
%reload_ext autoreload
%autoreload 2
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from utilities import data_handler
import numpy as np
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer # for tokenization only



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# THE CELL BELLOW CONTAINS THE FILENAMES TO CHANGE

In [2]:
UNLABLED_DATA = '/Users/ianmagnusson/IITUDND/data/retrieved_data/tweets/maria_extras.json'
LABLED_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/json/hurricane_maria_final_data.json'
CLASS_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/annotations/hurricane_maria_final_data.tsv'
NPY_OUTPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/maria/kfold/'

# get tweets from file and split into test/train

In [3]:
data = data_handler.DataHandler(UNLABLED_DATA,LABLED_DATA,CLASS_DATA)

_, validation_sets = data.get_k_fold_split(10)

 #test_labeled, test_histories, test_histories_by_target, train_merged, train_classes, test_classes = data.get_train_test_split()



## Set up glove

In [4]:
conversion_file = '../models/gensim_glove.txt'
# convert glove format to work with gensim. tutorial here https://radimrehurek.com/gensim/scripts/glove2word2vec.html
# _ = glove2word2vec('/Users/ianmagnusson/IITUDND/models/glove.twitter.27B.200d.txt', conversion_file)

In [5]:
# load model, NOTE this is very slow!
glove = KeyedVectors.load_word2vec_format(conversion_file)

# Extract features


In [6]:
# CODE FOR EXTRACTIONS

EMBED_DIM = 200

def embed_tweets(tweet_jsons):
    X_embedded = np.zeros((len(tweet_jsons),EMBED_DIM))
    tokenizer = CountVectorizer().build_tokenizer()
    for i, tweet_json in enumerate(tweet_jsons):
        text = tweet_json['text'].lower()
        tokens = [token for token in tokenizer(text) if token not in ENGLISH_STOP_WORDS]
        num_in_vocab = 0
        for token in tokens:
            if token in glove:
                X_embedded[i] += glove[token]
                num_in_vocab += 1
        X_embedded[i] = X_embedded[i] / num_in_vocab
    return X_embedded

def embed_histories(histories):
    X_embedded = np.zeros((len(histories),EMBED_DIM))
    tokenizer = CountVectorizer().build_tokenizer()
    for i, history in enumerate(histories):
        text = ' '.join([tweet_json['text'].lower() for tweet_json in history])
        tokens = [token for token in tokenizer(text) if token not in ENGLISH_STOP_WORDS]
        num_in_vocab = 0
        for token in tokens:
            if token in glove:
                X_embedded[i] += glove[token]
                num_in_vocab += 1
        X_embedded[i] = X_embedded[i] / num_in_vocab
    return X_embedded

def proccess_seq(histories_by_target):
    X_seq = [] # a list of 2d tensors of shape (len(seq), embed_dim)
    for history in histories_by_target:
        X_seq.append(embed_tweets(history))
    
    return X_seq

SVD_COMPONENTS = 200

def construct_vectorizer_and_SVD(merged):
    allTweets = []
    for i, tweet_json in enumerate(merged):
        text = tweet_json['text'].lower()
        allTweets.append(text)
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_merged = vectorizer.fit_transform(allTweets)
    svd = TruncatedSVD(n_components=SVD_COMPONENTS, n_iter=7, random_state=42)
    svd.fit(tfidf_merged)
    return vectorizer, svd


def vectorize_histories(histories, vectorizer, svd):
    rawHistories = [] # Will be in order
    for i, history in enumerate(histories):
        text = ' '.join([tweet_json['text'].lower() for tweet_json in history])
        rawHistories.append(text)
    histArr = vectorizer.transform(rawHistories)
    histFeatureArr = svd.transform(histArr)
    return histFeatureArr


def vectorize_tweets(tweets, vectorizer, svd):
    labeledTweets = []  # Will be in order
    for i, tweet_json in enumerate(tweets):
        text = tweet_json['text'].lower()
        labeledTweets.append(text)
    tweetArr = vectorizer.transform(labeledTweets)
    tweetFeatureArr = svd.transform(tweetArr)
    return tweetFeatureArr

def proccess_seq_tfidf(histories_by_target, vectorizer, svd):
    X_seq = [] # a list of 2d tensors of shape (len(seq), SVD_COMPONENTS)
    for history in histories_by_target:
        X_seq.append(vectorize_tweets(history, vectorizer, svd))
    return X_seq

In [7]:
for i, val_set in enumerate(validation_sets):
    print(i)
    
    out_dir = NPY_OUTPUT_DIR + str(i) + '/'
    train_labeled, train_histories, train_histories_by_target, train_classes, train_merged = val_set[0]
    test_labeled, test_histories, test_histories_by_target, test_classes, _ = val_set[1]
    
    # get data
    # classes:
    y_train = np.array(train_classes)
    y_test = np.array(test_classes)
    
    # glove baseline: 
    X_labeled_train = embed_tweets(train_labeled)
    X_histories_train = embed_histories(train_histories)
    X_labeled_test = embed_tweets(test_labeled)
    X_histories_test = embed_histories(test_histories)
    
    # glove sequence features
    X_seq_train = proccess_seq(train_histories_by_target)
    X_seq_test = proccess_seq(test_histories_by_target)
    
    # tfidf baseline
    vectorizer, svd = construct_vectorizer_and_SVD(train_merged)
    trainHistories = vectorize_histories(train_histories, vectorizer, svd)
    trainTweets = vectorize_tweets(train_labeled, vectorizer, svd)
    testHistories = vectorize_histories(test_histories, vectorizer, svd)
    testTweets = vectorize_tweets(test_labeled, vectorizer, svd)
    
    # tfidf sequence features
    X_seq_tfidf_train = proccess_seq_tfidf(train_histories_by_target, vectorizer, svd)
    X_seq_tfidf_test = proccess_seq_tfidf(test_histories_by_target, vectorizer, svd)
    
    
    # save data
    np.save(out_dir + 'y_train.npy', y_train)
    np.save(out_dir + 'y_test.npy', y_test)
    
    np.save(out_dir + 'X_labeled_train.npy', X_labeled_train)
    np.save(out_dir + 'X_histories_train.npy', X_histories_train)
    np.save(out_dir + 'X_labeled_test.npy', X_labeled_test)
    np.save(out_dir + 'X_histories_test.npy', X_histories_test)

    np.savez(out_dir + 'X_seq_glove_train.npz', *X_seq_train)
    np.savez(out_dir + 'X_seq_glove_test.npz', *X_seq_test)
    
    np.save(out_dir + 'trainHistories.npy', trainHistories)
    np.save(out_dir + 'trainTweets.npy', trainTweets)
    np.save(out_dir + 'testHistories.npy', testHistories)
    np.save(out_dir + 'testTweets.npy', testTweets)
    np.save(out_dir + 'trainClassifications.npy', train_classes)
    np.save(out_dir + 'testClassifications.npy', test_classes)
    
    np.savez(out_dir + 'X_seq_tfidf_train.npz', *X_seq_tfidf_train)
    np.savez(out_dir + 'X_seq_tfidf_test.npz', *X_seq_tfidf_test)
    






0
1
2
3
4
5
6
7
8
9
